In [1]:
import mne
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.metrics import Precision, Recall

In [2]:
import os

print(os.listdir("./data/"))
filepath = './data/p002_MI_withbins.set'

['p002_MI_markers_elist_f_cf pruned with ICA_bins_impel_be.fdt', 'p002_MI_markers_elist_f_cf pruned with ICA_bins_impel_be.set', 'p002_MI_markers_elist_f_cf pruned with ICA_bins_impel_be_ar.fdt', 'p002_MI_markers_elist_f_cf pruned with ICA_bins_impel_be_ar.set', 'p002_MI_withbins.set', 'test01.txt', 'test04.txt']


In [3]:
epochs = mne.read_epochs_eeglab(filepath)
df = epochs.to_data_frame()

Extracting parameters from D:\Development\EEG ANALITICS\data\p002_MI_withbins.set...
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
Ready.


C:\Users\Kiselev\AppData\Local\Temp\1\ipykernel_10988\2379057301.py:1: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  epochs = mne.read_epochs_eeglab(filepath)


In [4]:
from scipy.io import loadmat

def mat_struct_to_dict(mat_struct_obj):
    return {field: getattr(mat_struct_obj, field) for field in mat_struct_obj._fieldnames}
eeglab_data = loadmat(filepath, struct_as_record=False, squeeze_me=True)

events_data = eeglab_data['event']
df['codelabel'] = pd.NA
for i, event in enumerate(events_data):
    label = mat_struct_to_dict(event)['codelabel']
    df.loc[i, "codelabel"] = label

In [5]:
df = df.dropna()
df["codelabel"]

0       s0
1       s0
2      stf
3       s0
4       s0
      ... 
457     s0
458    stl
459     s0
460     s0
461     s0
Name: codelabel, Length: 462, dtype: object

In [28]:
# двигает рукой
stf = []
# частично двигает рукой
sts = []
# не двигает рукой - но представляет
stl = []
# не двигает вообще
s0 = []
for index, row in df.iterrows():
  if 'stf' in row['codelabel']:
   stf.append(index)
  elif 'sts' in row['codelabel']:
    sts.append(index)
  elif 'stl' in row['codelabel']:
    stl.append(index)
  elif 's0' in row['codelabel']:
    s0.append(index)
s0 = s0[:20]

In [29]:
df2 = df.drop('codelabel', axis=1)
df2['codelabel'] = pd.NA
for el in s0:
  df2.at[el, 'codelabel'] = 0
for el in stf:
  df2.at[el, 'codelabel'] = 1
for el in sts:
  df2.at[el, 'codelabel'] = 2
for el in stl:
  df2.at[el, 'codelabel'] = 3

In [30]:
df2 = df2.dropna()
df2["codelabel"]

0      0
1      0
2      1
3      0
4      0
      ..
434    3
440    3
446    3
452    3
458    3
Name: codelabel, Length: 110, dtype: object

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df2.drop(columns=['epoch','time','condition', 'codelabel'])
y = df2['codelabel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input

# Создание нейронной сети
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучение модели
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.0715 - loss: 1.7807 - val_accuracy: 0.5152 - val_loss: 1.3469
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.3602 - loss: 1.4387 - val_accuracy: 0.6667 - val_loss: 1.1515
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6138 - loss: 1.1395 - val_accuracy: 0.8485 - val_loss: 0.9975
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9233 - loss: 0.9227 - val_accuracy: 0.8788 - val_loss: 0.8866
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9129 - loss: 0.7498 - val_accuracy: 0.8788 - val_loss: 0.8080
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8531 - loss: 0.6438 - val_accuracy: 0.8182 - val_loss: 0.7479
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8179 - loss: 0.5896 - val_accuracy: 0.7879 - val_loss: 0.7016
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8010 - loss: 0.5276 - val_accuracy: 0.7576 - val_loss: 0.6697


In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model_tree = SVC(kernel='linear') 
model_tree.fit(X_train, y_train)
y_pred = model_tree.predict(X_test)
# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Дополнительные метрики
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.94
Confusion Matrix:
[[8 0 0 0]
 [1 7 0 0]
 [1 0 9 0]
 [0 0 0 7]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       1.00      0.88      0.93         8
           2       1.00      0.90      0.95        10
           3       1.00      1.00      1.00         7

    accuracy                           0.94        33
   macro avg       0.95      0.94      0.94        33
weighted avg       0.95      0.94      0.94        33



In [35]:
loss, accuracy = model.evaluate(X_test, y_test)

print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8163 - loss: 0.6114
Loss: 0.6126
Accuracy: 0.8182


In [36]:
model.save('model.keras')

In [37]:
def get_act_name(code):
    if code  == 0:
        return "s0"
    elif code == 1:
        return "stf"
    elif code == 2:
        return "sts"
    elif code == 3:
        return "stl"

In [38]:
test_num = 19
predicted = get_act_name(np.argmax(model.predict(X_test[test_num].reshape(1, 24))))
correct = get_act_name(list(y_test)[test_num])
print(f"predicted num is `{predicted}` and correct num is `{correct}`")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
predicted num is `stf` and correct num is `stf`


In [39]:
# # Загрузка модели
# model = tf.keras.models.load_model('model.keras')

# Инициализация конвертера
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Конвертация в формат TFLite
tflite_model = converter.convert()

# Сохранение TFLite модели в файл
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Kiselev\AppData\Local\Temp\1\tmpy5lybf7p\assets


INFO:tensorflow:Assets written to: C:\Users\Kiselev\AppData\Local\Temp\1\tmpy5lybf7p\assets


Saved artifact at 'C:\Users\Kiselev\AppData\Local\Temp\1\tmpy5lybf7p'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 24), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2098779292240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2098896029296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2098895942400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2098895944336: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [40]:
file_size = os.path.getsize("./model.tflite")

# Вывод размера файла
print(f"Размер файла: {file_size} байт")

Размер файла: 16576 байт


In [41]:
model_path= "model.tflite"

interpreter = tf.lite.Interpreter(model_path=model_path)


interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
print(input_details[0]['shape'])
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

[ 1 24]


In [42]:
#input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
input_data = X_test[19]
print(f"data on input: {input_data}")
print(f"It has action: {get_act_name(list(y_test)[19])}")
input_reshaped = input_data.reshape(input_shape)

# входные данные
interpreter.set_tensor(input_details[0]['index'], input_reshaped)
print("Input details:", input_details)
#  предсказание
interpreter.invoke()

# выходные данные
output_data = interpreter.get_tensor(output_details[0]['index'])

print("Predicted output:", output_data)
index_max = np.argmax(output_data)
print(get_act_name(list(y_test)[19]))

data on input: [-0.52109855 -0.57133687 -0.481523   -0.5882358  -0.80570453 -0.03607523
 -0.8392386  -0.8549552   0.3732829  -0.5899839   0.38921237  0.5864193
  1.0258859  -0.41387412 -0.5701152  -0.6521362  -0.8420634  -0.68346435
 -0.8151803  -0.668313    0.6052867   1.0809759   0.77627265 -0.77486044]
It has action: stf
Input details: [{'name': 'serving_default_keras_tensor_8:0', 'index': 0, 'shape': array([ 1, 24]), 'shape_signature': array([-1, 24]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Predicted output: [[0.14094679 0.65786195 0.15178901 0.04940225]]
1


In [43]:
np.savetxt('X_test.csv', X_test, delimiter=',', fmt='%f', header='', comments='')

In [ ]:
np.savetxt('Y_test.csv', y_test, delimiter=',', fmt='%f', header='', comments='')